# <a id='toc1_'></a>[Loan Default Prediction (Training)](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Loan Default Prediction (Training)](#toc1_)    
  - [Libraries](#toc1_1_)    
  - [Read Preprocessed data](#toc1_2_)    
    - [Transform](#toc1_2_1_)    
      - [Scaler](#toc1_2_1_1_)    
      - [One hot encoding](#toc1_2_1_2_)    
  - [Train](#toc1_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Libraries](#toc0_)

In [1]:
# Basic python
import numpy as np

# Data manipulation
import pandas as pd
import datetime as dt

# Transform
from sklearn.preprocessing import OneHotEncoder

## <a id='toc1_2_'></a>[Read Preprocessed data](#toc0_)

In [ ]:
# Define file path
path = "../data/raw/"
X_train_path, y_train_path = 
X_test_path, y_test_path = 

In [ ]:
# Read data
df = pl.read_csv(path + raw_file)


### <a id='toc1_2_1_'></a>[Transform](#toc0_)

#### <a id='toc1_2_1_1_'></a>[Scaler](#toc0_)

## <a id='toc1_3_'></a>[Train](#toc0_)